In [4]:
import sys
!{sys.executable} -m pip install implicit

In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [152]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [153]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? Зачем он нужен. Используя этот товар мы смещаем качество рекомендаций. В какую сторону?  Можно ли удалить этот товар? Уберите этот товар и сравните с качеством на семинаре.

### Ответ 0. Товар 999999
это фейковый популярный товар, он как бы группирует пользователей, не участвующих в покупке топ-5000, что позволяет снизить разреженность данных и ускорить обучение и подсчет. при этом так как данный товар попадает в топ он смещает рекомендации в худшую сторону. 

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекомендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [154]:
items_weights = pd.DataFrame(data.item_id.value_counts(normalize=True).reset_index().values, 
                             columns=['item_id', 'weight'])
items_weights.weight.sum()

0.9999999999999998

In [155]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(np.arange(len(items_weights)), size=n, p=np.array(items_weights.weight), replace=False)
    
    return recs.tolist()

Сделайте предсказания

In [156]:
%%time

# your_code
weighted_random_recommendation(items_weights, n=5)

CPU times: user 2.8 ms, sys: 3.52 ms, total: 6.32 ms
Wall time: 8.64 ms


[38195, 747, 29007, 1447, 5121]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [197]:
result = pd.read_csv('predictions_basic.csv', index_col=False) # закгрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[964594, 879948, 1066097, 6979803, 978313]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1127831, 995242]","[1082185, 99999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 99999999, 1098066]","[99999999, 1029743, 995785, 866227, 1110764]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1051283, 1100728, 999971, 999391, 9926758]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 99999999, 826249]","[1082185, 981760, 1098066, 826249, 99999999]","[99999999, 1023958, 8205731]"


In [185]:
result.reset_index()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[964594, 879948, 1066097, 6979803, 978313]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1127831, 995242]","[1082185, 99999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 99999999, 1098066]","[99999999, 1029743, 995785, 866227, 1110764]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1051283, 1100728, 999971, 999391, 9926758]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 99999999, 826249]","[1082185, 981760, 1098066, 826249, 99999999]","[99999999, 1023958, 8205731]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[849297, 925425, 947068, 947858, 831517]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1127831, 995242]","[1082185, 99999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 99999999, 878996]","[99999999, 1029743, 995785, 866227, 1046262]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[916758, 1055503, 987557, 853354, 937928]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 99999999, 1127831, 961554]","[1082185, 981760, 1127831, 99999999, 961554]","[99999999, 1029743, 995785, 1046262]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[932242, 1103752, 831390, 864312, 844441]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 99999999, 1098066, 826249]","[1082185, 981760, 99999999, 1098066, 826249]","[99999999, 1029743, 995785, 1046262, 1034176]"
...,...,...,...,...,...,...,...,...
2037,2496,[6534178],"[1137387, 895680, 1029737, 1024051, 903567]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 99999999, 1098066, 826249]","[1082185, 981760, 1098066, 99999999, 826249]","[99999999, 995785, 866227, 901747]"
2038,2497,[ 1016709 9835695 1132298 16809501 845294 ...,"[876806, 864033, 1096099, 908938, 936508]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1098066, 1127831]","[1082185, 981760, 99999999, 1098066, 826249]","[1082185, 981760, 1098066, 99999999, 826249]","[99999999, 1029743, 995785, 942525, 1110764]"
2039,2498,[15716530 834484 901776 914190 958382 ...,"[7409971, 929751, 6919433, 1011141, 5591038]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1098066, 1127831]","[1082185, 99999999, 981760, 1098066, 826249]","[1082185, 981760, 99999999, 1098066, 826249]","[99999999, 1029743, 866227]"
2040,2499,[ 867188 877580 902396 914190 951590 ...,"[6551782, 869857, 847637, 5567486, 12263614]","[99999999, 6534178, 6533889, 1029743, 6534166]","[99999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 99999999, 1098066, 826249]","[1082185, 981760, 1098066, 99999999, 826249]","[99999999, 866227, 910291]"


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [ ]:
# your_code

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.